<a href="https://colab.research.google.com/github/Padhul/tiket-Dies-Natalis/blob/main/Python_Loket.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gradio gTTS


In [4]:
!pip install --upgrade gradio


In [3]:
!pip install gradio reportlab


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.1 MB/s eta 0:00:00


In [2]:
!pip install fpdf


  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=6b7b59e948720c2b912cb3cf8ce905fb0b5a97639b5109ff652d73ad613aedfc
  Stored in directory: /root/.cache/pip/wheels/6e/62/11/dc73d78e40a218ad52e7451f30166e94491be013a7850b5d75
Successfully built fpdf


In [2]:
# === PENGGUNA — Ambil Nomor (menyimpan ke data_antrian.json) ===
# Colab-friendly, tidak perlu DatePicker. Optional PDF generation jika reportlab terinstall.

import gradio as gr
import json, os, time
from datetime import date, datetime

DATA_FILE = "data_antrian.json"
PDF_FOLDER = "pdf_tickets"
os.makedirs(PDF_FOLDER, exist_ok=True)

# ---------- DB helpers ----------
def load_data():
    """Load JSON DB; ensure keys exist and return dictionary."""
    if not os.path.exists(DATA_FILE):
        data = {
            "nomor_terakhir": 0,
            "antrian": [],
            "daftar_waktu_layanan": [],
            "sedang_dilayani": {1: 0, 2: 0},
            "waktu_mulai": {1: None, 2: None},
            "records": []
        }
        save_data(data)
        return data
    with open(DATA_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

def save_data(data):
    with open(DATA_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

# ---------- Optional PDF generator (reportlab) ----------
def generate_ticket_pdf(nomor, nama, kelas, tanggal_str):
    """Create a small PDF ticket using reportlab if available.
       Returns path to PDF or None if reportlab not installed / error."""
    try:
        from reportlab.lib.pagesizes import A6, landscape
        from reportlab.pdfgen import canvas
    except Exception:
        # reportlab not available
        return None

    fname = f"ticket_{nomor}.pdf"
    path = os.path.join(PDF_FOLDER, fname)
    width, height = landscape(A6)
    c = canvas.Canvas(path, pagesize=(width, height))

    # Simple styled ticket
    c.setFont("Helvetica-Bold", 16)
    c.drawCentredString(width/2, height - 30, "TIKET ANTRIAN")
    c.setFont("Helvetica-Bold", 36)
    c.drawCentredString(width/2, height - 70, f"#{nomor}")
    c.setFont("Helvetica", 10)
    c.drawCentredString(width/2, height - 100, f"Tanggal: {tanggal_str}")

    left_x = 18
    y = height - 140
    line_gap = 14
    c.setFont("Helvetica-Bold", 10)
    c.drawString(left_x, y, "Nama")
    c.setFont("Helvetica", 12)
    c.drawString(left_x + 60, y, nama)
    y -= line_gap
    c.setFont("Helvetica-Bold", 10)
    c.drawString(left_x, y, "Kelas")
    c.setFont("Helvetica", 12)
    c.drawString(left_x + 60, y, kelas)

    # Footer timestamp
    c.setFont("Helvetica-Oblique", 7)
    ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    c.drawRightString(width - 8, 8, f"Dicetak: {ts}")

    c.showPage()
    c.save()
    return path

# ---------- Main action ----------
def ambil_nomor_dan_cetak(nama, kelas, tanggal):
    """
    Inputs:
      - nama: str
      - kelas: str
      - tanggal: str (format YYYY-MM-DD) or empty
    Outputs:
      - markdown status
      - file path for PDF (or None)
    """
    # sanitize
    nama = (nama or "").strip()
    kelas = (kelas or "").strip()
    tanggal_input = (tanggal or "").strip()

    if not nama or not kelas:
        return "❌ Mohon isi Nama dan Kelas.", None

    # determine tanggal string
    if not tanggal_input:
        tanggal_str = date.today().isoformat()
    else:
        tanggal_str = tanggal_input

    # load DB, update nomor & antrian
    db = load_data()
    db["nomor_terakhir"] = int(db.get("nomor_terakhir", 0)) + 1
    nomor_baru = db["nomor_terakhir"]

    # append to queue
    db.setdefault("antrian", []).append(nomor_baru)

    # append record metadata
    rec = {
        "nomor": nomor_baru,
        "nama": nama,
        "kelas": kelas,
        "tanggal": tanggal_str,
        "created_at": datetime.now().isoformat()
    }
    db.setdefault("records", []).append(rec)

    # save to JSON
    save_data(db)

    # Try to generate PDF (optional)
    pdf_path = generate_ticket_pdf(nomor_baru, nama, kelas, tanggal_str)

    md = (
        f"### ✅ Nomor Antrian Anda: **{nomor_baru}**\n\n"
        f"- Nama: **{nama}**\n"
        f"- Kelas: **{kelas}**\n"
        f"- Tanggal: **{tanggal_str}**\n\n"
        f"Terlampir tiket (PDF) bila fungsi PDF tersedia."
    )

    if pdf_path:
        return md, pdf_path
    else:
        if os.path.exists("reportlab"):
            # unlikely, but handle
            pass
        note = "\n\n⚠️ PDF tidak dibuat (library `reportlab` tidak ditemukan). Jika ingin PDF, jalankan `!pip install reportlab` lalu ulangi."
        return md + note, None

# ---------- Build Gradio UI ----------
with gr.Blocks(title="Pengguna — Ambil Nomor (Terhubung ke Operator)") as demo_user:
    gr.Markdown("## 🎫 Pengguna — Ambil Nomor\nIsi Nama & Kelas; tanggal opsional.")
    with gr.Row():
        with gr.Column():
            nama_in = gr.Textbox(label="Nama", placeholder="Masukkan nama lengkap")
            kelas_in = gr.Textbox(label="Kelas", placeholder="Contoh: XI-IPA 1")
            tanggal_in = gr.Textbox(label="Tanggal (opsional, YYYY-MM-DD)", placeholder="kosong = hari ini")
            btn = gr.Button("🎟️ Ambil Nomor & Cetak (opsional PDF)")
            output_md = gr.Markdown("Hasil akan muncul di sini")
            file_out = gr.File(label="Unduh PDF (jika dibuat)")
        with gr.Column():
            gr.Markdown("**Catatan:** aplikasi ini menulis ke `data_antrian.json`.\nOperator yang menjalankan panel operator harus berada pada folder yang sama untuk melihat update.\nJika Anda ingin PDF, install reportlab: `!pip install reportlab`")

    btn.click(fn=ambil_nomor_dan_cetak, inputs=[nama_in, kelas_in, tanggal_in], outputs=[output_md, file_out])

# launch
if __name__ == "__main__":
    demo_user.launch(debug=True)


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7cc542b6b0e6f718c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://7cc542b6b0e6f718c1.gradio.live


In [3]:
# === Operator panel — debug + auto-load saat start ===
import gradio as gr
import time, os, json
from gtts import gTTS

DATA_FILE = "data_antrian.json"
AUDIO_FOLDER = "audio_antrian"
os.makedirs(AUDIO_FOLDER, exist_ok=True)

# ---------- DB helpers ----------
def load_db():
    if not os.path.exists(DATA_FILE):
        db = {
            "nomor_terakhir": 0,
            "antrian": [],
            "daftar_waktu_layanan": [],
            "sedang_dilayani": {1: 0, 2: 0},
            "waktu_mulai": {1: None, 2: None}
        }
        save_db(db)
        return db
    with open(DATA_FILE, "r") as f:
        raw = json.load(f)
    antrian = raw.get("antrian", [])
    daftar = raw.get("daftar_waktu_layanan", [])
    tmp_sd = raw.get("sedang_dilayani", {"1": 0, "2": 0})
    tmp_waktu = raw.get("waktu_mulai", {"1": None, "2": None})
    sedang_dilayani = {1: tmp_sd.get("1", tmp_sd.get(1, 0)), 2: tmp_sd.get("2", tmp_sd.get(2, 0))}
    waktu_mulai = {1: tmp_waktu.get("1", tmp_waktu.get(1, None)), 2: tmp_waktu.get("2", tmp_waktu.get(2, None))}
    return {
        "nomor_terakhir": raw.get("nomor_terakhir", 0),
        "antrian": antrian,
        "daftar_waktu_layanan": daftar,
        "sedang_dilayani": sedang_dilayani,
        "waktu_mulai": waktu_mulai
    }

def save_db(db):
    to_save = {
        "nomor_terakhir": db.get("nomor_terakhir", 0),
        "antrian": list(db.get("antrian", [])),
        "daftar_waktu_layanan": db.get("daftar_waktu_layanan", []),
        "sedang_dilayani": {str(k): v for k, v in db.get("sedang_dilayani", {1:0,2:0}).items()},
        "waktu_mulai": {str(k): v for k, v in db.get("waktu_mulai", {1:None,2:None}).items()}
    }
    with open(DATA_FILE, "w") as f:
        json.dump(to_save, f)

# ---------- Utilities ----------
def format_waktu(detik):
    if detik is None:
        return "-"
    if detik < 60:
        return f"{int(detik)} detik"
    menit = int(detik // 60)
    sisa = int(detik % 60)
    return f"{menit} menit {sisa} detik"

def hitung_rata(daftar):
    return sum(daftar) / len(daftar) if daftar else 0

def render_queue_html_str(antrian):
    if not antrian:
        return "<div><i>Tidak ada antrian</i></div>"
    html = "<div style='font-family:system-ui;padding:8px'><ol>"
    for i, n in enumerate(antrian, start=1):
        html += f"<li><b>#{n}</b> <small style='color:gray'>(pos {i})</small></li>"
    html += "</ol></div>"
    return html

def render_stats_html_str(db):
    arr = db.get("daftar_waktu_layanan", [])
    total = len(arr)
    rata = hitung_rata(arr)
    tercepat = min(arr) if arr else 0
    terlama = max(arr) if arr else 0
    html = "<div style='font-family:system-ui;padding:8px'>"
    html += f"<div><b>Total terlayani:</b> {total}</div>"
    html += f"<div><b>Rata-rata:</b> {format_waktu(rata) if total>0 else '-'}</div>"
    html += f"<div style='margin-top:8px'><b>Loket:</b></div>"
    for lok in [1,2]:
        st = db["sedang_dilayani"].get(lok, 0)
        html += f"<div>Loket {lok}: {('Nomor '+str(st)) if st!=0 else '<span style=color:gray>kosong</span>'}</div>"
    html += f"<div style='margin-top:8px; color:gray; font-size:smaller;'>Tercepat: {format_waktu(tercepat) if total>0 else '-'} — Terlama: {format_waktu(terlama) if total>0 else '-'}</div>"
    html += "</div>"
    return html

# ---------- TTS ----------
def panggil_nomor_audio(nomor, loket):
    filename = os.path.join(AUDIO_FOLDER, f"nomor_{nomor}_loket_{loket}.mp3")
    if not os.path.exists(filename):
        try:
            text = f"Nomor {nomor}, silakan menuju loket {loket}."
            tts = gTTS(text=text, lang='id')
            tts.save(filename)
            print("[debug] TTS dibuat:", filename)
        except Exception as e:
            print("TTS error:", e)
            return None
    return filename

# ---------- Operator actions (return raw strings/html) ----------
def op_status():
    db = load_db()
    print("[debug] op_status - db loaded:", db)           # debug
    info = f"Antrian: {db['antrian'] if db['antrian'] else 'kosong'}\nSedang dilayani: Loket1={db['sedang_dilayani'].get(1,0)}, Loket2={db['sedang_dilayani'].get(2,0)}"
    return info, render_queue_html_str(db["antrian"]), render_stats_html_str(db)

def op_call(loket):
    db = load_db()
    print("[debug] op_call - before:", db)                # debug
    lok = int(loket)
    if db["sedang_dilayani"].get(lok, 0) != 0:
        return f"⚠️ Loket {lok} sedang melayani nomor {db['sedang_dilayani'][lok]}", render_queue_html_str(db["antrian"]), render_stats_html_str(db)
    if not db["antrian"]:
        return "ℹ️ Antrian kosong", render_queue_html_str(db["antrian"]), render_stats_html_str(db)
    nomor = db["antrian"].pop(0)
    db["sedang_dilayani"][lok] = nomor
    db["waktu_mulai"][lok] = time.time()
    save_db(db)
    audio_rel = panggil_nomor_audio(nomor, lok)
    msg = f"🔔 NOMOR {nomor}, silakan menuju Loket {lok}!"
    print("[debug] op_call - after:", db, "audio:", audio_rel)  # debug
    # IMPORTANT: gr.Audio expects a path string (or None). We'll return audio_rel as third output if present.
    # Return order: status_text, queue_html (str), stats_html (str), audio_path (string or None), fallback_html (str)
    fallback_html = f'<audio controls src="{audio_rel}"></audio>' if audio_rel else ""
    return msg, render_queue_html_str(db["antrian"]), render_stats_html_str(db), audio_rel, fallback_html

def op_complete(loket):
    db = load_db()
    print("[debug] op_complete - before:", db)            # debug
    lok = int(loket)
    curr = db["sedang_dilayani"].get(lok, 0)
    if curr == 0:
        return f"ℹ️ Tidak ada layanan di Loket {lok}", render_queue_html_str(db["antrian"]), render_stats_html_str(db)
    start = db["waktu_mulai"].get(lok)
    dur = (time.time() - start) if start else 0
    db["daftar_waktu_layanan"].append(dur)
    db["sedang_dilayani"][lok] = 0
    db["waktu_mulai"][lok] = None
    save_db(db)
    print("[debug] op_complete - after:", db)             # debug
    return f"✔️ Selesai: Nomor {curr}. Waktu layanan: {format_waktu(dur)}", render_queue_html_str(db["antrian"]), render_stats_html_str(db)

def op_reset_execute():
    if os.path.exists(DATA_FILE):
        os.remove(DATA_FILE)
    db = {
        "nomor_terakhir": 0,
        "antrian": [],
        "daftar_waktu_layanan": [],
        "sedang_dilayani": {1: 0, 2: 0},
        "waktu_mulai": {1: None, 2: None}
    }
    save_db(db)
    print("[debug] op_reset_execute - db reset")          # debug
    return "🗑️ Data berhasil direset!", render_queue_html_str(db["antrian"]), render_stats_html_str(db)

# ---------- Build Gradio UI with demo.load to populate initial UI ----------
with gr.Blocks(title="Operator Panel — Sistem Antrian (debug + load)") as demo:
    gr.Markdown("## 🧑‍💼 Operator — Panggil Loket (TTS MP3) & Statistik")
    with gr.Row():
        with gr.Column(scale=1):
            btn_call_1 = gr.Button("🔊 Panggil Loket 1")
            btn_done_1 = gr.Button("✅ Selesai Loket 1")
            btn_call_2 = gr.Button("🔊 Panggil Loket 2")
            btn_done_2 = gr.Button("✅ Selesai Loket 2")
            btn_refresh = gr.Button("🔁 Refresh Status")
            btn_reset = gr.Button("⚠️ Reset Data")
        with gr.Column(scale=2):
            status_out = gr.Textbox(interactive=False, label="Status Panggilan / Info")
            audio_out = gr.Audio(label="Audio Panggilan (klik play jika autoplay diblokir)", autoplay=False)
            audio_fallback = gr.HTML("")
            queue_html = gr.HTML("<i>Memuat antrian...</i>")
            stats_html = gr.HTML("<i>Memuat statistik...</i>")

    # demo.load populates UI at start
    demo.load(fn=op_status, outputs=[status_out, queue_html, stats_html])

    # Bindings
    btn_refresh.click(fn=op_status, outputs=[status_out, queue_html, stats_html])
    # op_call returns: status_text, queue_html, stats_html, audio_path, fallback_html
    btn_call_1.click(fn=lambda: op_call(1), outputs=[status_out, queue_html, stats_html, audio_out, audio_fallback])
    btn_call_2.click(fn=lambda: op_call(2), outputs=[status_out, queue_html, stats_html, audio_out, audio_fallback])
    btn_done_1.click(fn=lambda: op_complete(1), outputs=[status_out, queue_html, stats_html])
    btn_done_2.click(fn=lambda: op_complete(2), outputs=[status_out, queue_html, stats_html])
    btn_reset.click(fn=op_reset_execute, outputs=[status_out, queue_html, stats_html])

# Launch
demo.launch(debug=True)



It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://d12b294daea8102f88.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[debug] op_status - db loaded: {'nomor_terakhir': 3, 'antrian': [1, 2, 3], 'daftar_waktu_layanan': [], 'sedang_dilayani': {1: 0, 2: 0}, 'waktu_mulai': {1: None, 2: None}}
[debug] op_call - before: {'nomor_terakhir': 3, 'antrian': [1, 2, 3], 'daftar_waktu_layanan': [], 'sedang_dilayani': {1: 0, 2: 0}, 'waktu_mulai': {1: None, 2: None}}
[debug] TTS dibuat: audio_antrian/nomor_1_loket_1.mp3
[debug] op_call - after: {'nomor_terakhir': 3, 'antrian': [2, 3], 'daftar_waktu_layanan': [], 'sedang_dilayani': {1: 1, 2: 0}, 'waktu_mulai': {1: 1763358764.8741436, 2: None}} audio: audio_antrian/nomor_1_loket_1.mp3
[debug] op_complete - before: {'nomor_terakhir': 3, 'antrian': [2, 3], 'daftar_waktu_layanan': [], 'sedang_dilayani': {1: 1, 2: 0}, 'waktu_mulai': {1: 1763358764.8741436, 2: None}}
[debug] op_complete - after: {'nomor_terakhir': 3, 'antrian': [2, 3], 'daftar_waktu_layanan': [10.440560817718506], 'sedang_dilayani': {1: 0, 2: 0}, 'waktu_mulai': {1: None, 2: None}}
[debug] op_call - before: {